In [465]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from fbprophet import Prophet
from datetime import datetime
from numpy import array
from numpy import split
import matplotlib.pyplot as plt

In [466]:
timestr = time.strftime("%Y_%m_%d")
timestr = '2020_04_14'
df = pd.read_csv(r'..\..\data\dev_data\covid19_time_series_{}.csv'.format(timestr))
##Active is always zero...manually correct via simple calculation
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']

## Multiple Linear Regression Approach

In [484]:
## really only need the date and confirmed columns but bringing along the other in case we want to predict any of them
country_df_regres = df.groupby(['Country','Date']).sum()[["Confirmed","Recovered","Active","Deaths"]].reset_index()
country_df_regres['Date'] = pd.to_datetime(country_df_regres['Date'])

In [485]:
## create lag variables from the last two weeks
for i in range(7, 11):
    country_df_regres[f'Confirmed_lag_{i}'] = country_df_regres.groupby('Country')['Confirmed'].shift(i)
    country_df_regres[f'Deaths_lag_{i}'] = country_df_regres.groupby('Country')['Deaths'].shift(i)
    country_df_regres[f'Recovered_lag_{i}'] = country_df_regres.groupby('Country')['Recovered'].shift(i)
    country_df_regres[f'Active_lag{i}'] = country_df_regres.groupby('Country')['Active'].shift(i)

In [486]:
country_list=country_df_regres.Country.unique()

In [488]:
mapes_linear_reg = []
cntries = []
for cntry in country_list:
    df_train = country_df_regres.loc[country_df_regres['Country']==cntry]
    df_train = df_train.dropna()
    df_train= df_train.loc[df_train['Confirmed_lag_7']>0]
    df_train = df_train.reset_index()
    lag_cols_only =  [col for col in df_train.columns if 'Confirmed_lag_' in col]
    X = df_train[lag_cols_only]
    y = df_train['Confirmed']
    X_train = X.iloc[0:len(X)-7]
    X_test = X.iloc[-7:]
    y_train = y.iloc[0:len(y)-7]
    y_test = y.iloc[-7:]
    
    if len(X) > 13:
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mapes_linear_reg.append((abs((y_pred -y_test) / y_test) * 100).mean())
        cntries.append(cntry)
    else:
        pass

In [489]:
len(cntries)

170

In [490]:
np.mean(mapes_linear_reg)

28.831548074412368

## Holt Linear Model

In [491]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [498]:
mapes_country_holt = []
for cntry in country_list:
    df_train = country_df_regres.loc[country_df_regres['Country']==cntry]
    df_train = df_train.dropna()
    df_train= df_train.loc[df_train['Confirmed_lag_7']>0]
    df_train = df_train.reset_index()
    cols=['Country','Date','Confirmed']
    lag_cols_only =  [col for col in df_train.columns if 'Confirmed_lag_' in col]
    X = df_train[lag_cols_only]
    y = df_train['Confirmed']
    y_preds_holt = []
    mapes_holt =  []

    if len(X) > 13:
        y_train = y.iloc[0:len(y)-7]
        y_test = y.iloc[-7:]
        holtfit = ExponentialSmoothing(y_train, trend='mul',seasonal=None,damped=True).fit()
        y_pred = holtfit.forecast(len(y_test))
        y_preds_holt.append(np.round(y_pred.values))
        mapes_country_holt.append((abs((y_pred.values -y_test) / y_test) * 100).mean())
    else:
        pass

C:\Users\pspat\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

C:\Users\pspat\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

C:\Users\pspat\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

C:\Users\pspat\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

C:\Users\pspat\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

C:\Users\pspat\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

C:\Users\pspat\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:744: Con

In [499]:
len(mapes_country_holt)

170

In [500]:
np.mean(mapes_country_holt)

19.205360475028694

In [353]:
mapes_country_arima = []
for cntry in country_list:
    df_train = country_df_regres.loc[country_df_regres['Country']==cntry]
    df_train = df_train.dropna()
    df_train= df_train.loc[df_train['Confirmed_lag_7']>0]
    df_train = df_train.reset_index()
    cols=['Country','Date','Confirmed']
    lag_cols_only =  [col for col in df_train.columns if 'Confirmed_lag_' in col]
    X = df_train[lag_cols_only]
    y = df_train['Confirmed']
    y_preds_arima = []

    if len(X) > 13:
        try:
            y_train = y.iloc[0:len(y)-7]
            y_test = y.iloc[-7:]
            model_arima=ARIMA(y_train,(2,2,0))
            model_arima_fit=model_arima.fit()
            y_pred = model_arima_fit.forecast(len(y_test))[0]
            mapes_country_arima.append((abs((y_pred -y_test) / y_test) * 100).mean())
        except:
            pass
    else:
        pass

C:\Users\pspat\Anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning:

Inverting hessian failed, no bse or cov_params available

C:\Users\pspat\Anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\pspat\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:1866: RuntimeWarning:

invalid value encountered in sqrt

C:\Users\pspat\Anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning:

Inverting hessian failed, no bse or cov_params available

C:\Users\pspat\Anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\pspat\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:670: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\pspat\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:672: Runtim

In [355]:
np.mean(mapes_country_arima)

13.424352906953903

In [379]:
mapes_country_proph = []
for cntry in country_list:
    df_cntry = country_df_regres.loc[country_df_regres['Country']==cntry]
    df_cntry = df_cntry.dropna()
    df_cntry= df_cntry.loc[df_cntry['Confirmed_lag_7']>0]
    df_cntry = df_cntry.reset_index()
    df_fb_prophet = df_cntry[['Date','Confirmed']]
    df_fb_prophet.rename(columns={'Date':'ds','Confirmed':'y'},inplace=True)
    df_fb_prophet['ds']=df_fb_prophet['ds'].apply(lambda x: x.date())
    
    if len(X) > 13:
        try:
            df_train = df_fb_prophet.iloc[0:len(df_fb_prophet)-7]
            prophet_model=Prophet()
            prophet_model.fit(df_train)
            future = prophet_model.make_future_dataframe(periods=7)
            forecast = prophet_model.predict(future)
            forecast=forecast[-7:]
            y_pred = forecast['yhat'].values
            y_test = df_fb_prophet.iloc[-7:]['y']
            mapes_country_proph.append(round((abs((y_pred -y_test) / y_test) * 100).mean()))
        
        except:
            pass
    else:
        pass

C:\Users\pspat\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 16.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run p

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 11.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 8.
INFO:fbprophet:Disabling yearly seasonality. Run 

INFO:fbprophet:n_changepoints greater than number of observations. Using 12.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 10.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disab

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 4.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 22.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabl

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 24.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 8.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabl

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 19.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 19.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 11.
INFO:fbprophet:Disabling yearly seasonality. Run 

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.


In [380]:
np.mean(mapes_country_proph)

21.565714285714286